<a href="https://colab.research.google.com/github/kys8411/AIFFEL_Exploration/blob/main/%5B%EC%9D%B4%EA%B8%B0%EC%B0%BD_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC%5D_5_2_%EB%AC%B8%EC%9E%A5_%EC%8C%8D_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기

TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [1]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

TPU 이외에 의존성 있는 패키지를 설치합니다.

In [ ]:
!pip install ratsnlp

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
## PyTorch와 RatsNLP 라이브러리를 사용하여 텍스트 분류 작업을 위한 설정을 구성
## 아래 설정을 통해 텍스트 분류 작업을 수행할 때 매개변수 설정
## 실제로 텍스트 분류 작업을 수행하려면 해당 매개변수를 모델 학습에 사용해야 함.


import torch
# PyTorch 라이브러리를 임포트합니다. PyTorch는 머신 러닝을 위한 오픈 소스 라이브러리입니다.

from ratsnlp.nlpbook.classification import ClassificationTrainArguments
# RatsNLP 라이브러리에서 ClassificationTrainArguments 클래스를 임포트합니다. 이 클래스는 텍스트 분류 작업을 위한 학습 매개변수를 설정할 때 사용합니다.

args = ClassificationTrainArguments(
    # 다음 매개변수들을 사용하여 ClassificationTrainArguments 클래스의 인스턴스를 생성하고, 이를 변수 args에 할당합니다.

    pretrained_model_name="beomi/kcbert-base",
    # 사전 학습된 모델의 이름을 지정합니다. 여기서는 "beomi/kcbert-base"라는 모델을 사용합니다. 이 모델은 한국어 텍스트에 대한 BERT 모델입니다.

    downstream_task_name="pair-classification",
    # 다운스트림 작업의 이름을 지정합니다. 여기서는 "pair-classification" 작업을 선택합니다.

    downstream_corpus_name="klue-nli",
    # 다운스트림 작업에 사용할 데이터셋의 이름을 지정합니다. 여기서는 "klue-nli"라는 데이터셋을 사용합니다.

    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    # 학습된 모델을 저장할 경로를 지정합니다.

    batch_size=32 if torch.cuda.is_available() else 4,
    # 배치 크기를 설정합니다. GPU가 사용 가능한 경우 배치 크기를 32로, 그렇지 않은 경우 4로 설정합니다.

    learning_rate=5e-5,
    # 학습률을 설정합니다.

    max_seq_length=64,
    # 입력 시퀀스의 최대 길이를 설정합니다.

    epochs=5,
    # 전체 데이터셋에 대해 학습을 반복할 횟수를 설정합니다.

    tpu_cores=0 if torch.cuda.is_available() else 8,
    # TPU 코어의 개수를 설정합니다. GPU가 사용 가능한 경우 TPU 코어 개수를 0으로, 그렇지 않은 경우 8로 설정합니다.

    seed=7,
    # 랜덤 시드 값을 설정합니다.
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
## 랜덤시드를 설정하는 이유는 실험의 재현성을 보장하기 위해서 (결과값이 동일한 결과)

from ratsnlp import nlpbook
#  ratsnlp 라이브러리의 nlpbook 모듈을 임포트합니다.

nlpbook.set_seed(args)
# nlpbook 모듈의 set_seed 함수를 사용하여 랜덤 시드를 설정합니다. 여기서 args는 앞서 설정한 ClassificationTrainArguments 클래스의 인스턴스입니다. 이 인스턴스 내에 seed라는 속성으로 랜덤 시드 값이 저장되어 있으며, set_seed 함수가 이 값을 사용하여 시드를 설정합니다.


set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [ ]:
## ratsnlp 라이브러리의 nlpbook 모듈에서 set_logger 함수를 사용하여 로거를 설정하는 과정
## 로거는 프로그램의 실행 중에 발생하는 이벤트를 기록하는데 사용됩니다. 이를 통해 학습 과정에서 발생하는 이벤트를 추적하고, 오류 발생 시에 원인 분석에 도움을 줄 수 있습니다. set_logger 함수는 args를 사용하여 로깅 관련 설정을 수행하며, 일반적으로 로깅 수준, 출력 형식, 출력 위치 등을 설정할 수 있습니다. 이를 통해 학습 과정에서 발생하는 이벤트를 효과적으로 기록하고 관리할 수 있습니다.

nlpbook.set_logger(args)
# nlpbook 모듈의 set_logger 함수를 사용하여 로깅 설정을 수행합니다. 여기서 args는 앞서 설정한 ClassificationTrainArguments 클래스의 인스턴스입니다.

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_siz

# 말뭉치 다운로드
실습에 사용할 말뭉치를 다운로드합니다.

In [ ]:
## ratsnlp 라이브러리의 nlpbook 모듈에서 download_downstream_dataset 함수를 사용하여 다운스트림 작업을 위한 데이터셋을 다운로드하는 과정을 보여줍니다.

nlpbook.download_downstream_dataset(args)
# nlpbook 모듈의 download_downstream_dataset 함수를 호출하여 다운스트림 작업에 필요한 데이터셋을 다운로드합니다. 여기서 args는 앞서 설정한 ClassificationTrainArguments 클래스의 인스턴스입니다.

# args에는 다운스트림 작업과 관련된 설정이 포함되어 있으며, 이 설정에 따라 해당 함수는 필요한 데이터셋을 다운로드합니다.
# 이 함수는 args의 downstream_corpus_name 속성을 사용하여 어떤 데이터셋을 다운로드해야 하는지 결정합니다.
# 이전에 설정한 args에서 downstream_corpus_name은 "klue-nli"로 설정되었습니다.
# 따라서 이 함수는 "klue-nli" 데이터셋을 다운로드합니다.
# 이 데이터셋은 한국어 자연어 추론(NLI) 작업에 사용되는 데이터셋입니다.

## 다운로드한 데이터셋은 학습 및 평가에 사용됩니다. 다운로드 경로와 관련 설정은 args 내의 속성을 통해 제어됩니다.

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 23.8MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 10.4MB/s]


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [ ]:
## transformers 라이브러리의 BertTokenizer 를 사용하여 토크나이저를 설정하는 코드

from transformers import BertTokenizer
# transformers 라이브러리에서 BertTokenizer 클래스를 임포트합니다.

tokenizer = BertTokenizer.from_pretrained(
    # BertTokenizer 클래스의 from_pretrained 메서드를 사용하여 사전 학습된 모델을 기반으로 토크나이저를 생성하고, 그 결과를 tokenizer 변수에 할당합니다. 이때, 사용하는 매개변수는 다음과 같습니다.

    args.pretrained_model_name,
    # 사용할 사전 학습된 모델의 이름을 지정합니다. 앞서 설정한 args의 pretrained_model_name 속성을 사용하므로, 이전에 설정한 "beomi/kcbert-base"라는 모델이 사용됩니다.

    do_lower_case=False,
    # 텍스트를 토큰화할 때 모든 문자를 소문자로 변환할지 여부를 지정합니다. 여기서는 False로 설정하여 문자를 소문자로 변환하지 않도록 합니다.
)

## 결과적으로 tokenizer 변수에는 사전 학습된 모델 "beomi/kcbert-base"에 대한 토크나이저가 설정됩니다. 이 토크나이저를 사용하여 텍스트를 토큰으로 분리하고, 토큰을 숫자로 변환할 수 있습니다. 이러한 토큰화 및 인코딩 작업은 텍스트 데이터를 모델이 처리할 수 있는 형식으로 변환하는데 필요합니다.

# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
## 코드는 데이터를 처리하고 학습을 위한 데이터 로더를 준비하는 과정을 보여줍니다.

from ratsnlp.nlpbook.paircls import KlueNLICorpus
#  ratsnlp 라이브러리에서 KlueNLICorpus 클래스를 가져옵니다. 이 클래스는 KLUE NLI 데이터셋을 처리하는 역할을 합니다.

from ratsnlp.nlpbook.classification import ClassificationDataset
# 분류 작업을 위한 데이터셋 클래스를 가져옵니다.

from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
# PyTorch에서 제공하는 데이터 로더와 샘플러를 가져옵니다.

corpus = KlueNLICorpus()
# KLUE NLI 데이터셋 객체를 생성합니다.

train_dataset = ClassificationDataset(
    # 학습용 데이터셋을 만듭니다.
    args=args,
    # 설정 값을 담고 있는 객체입니다.
    corpus=corpus,
    # 방금 생성한 KLUE NLI 데이터셋 객체입니다.
    tokenizer=tokenizer,
    # BERT 토크나이저입니다.
    mode="train",
    # 이 데이터셋이 학습용임을 나타냅니다.
)
train_dataloader = DataLoader(
    # 데이터 로더를 설정합니다.
    train_dataset,
    # 학습용 데이터셋입니다.
    batch_size=args.batch_size,
    # 한 번에 처리할 데이터의 양입니다.
    sampler=RandomSampler(train_dataset, replacement=False),
    # 데이터를 무작위로 선택하는 방식입니다.
    collate_fn=nlpbook.data_collator,
    # 데이터를 한 번에 모아주는 함수입니다.
    drop_last=False,
    # 마지막에 남은 데이터도 사용하도록 설정합니다.
    num_workers=args.cpu_workers,
    # 데이터를 로드할 때 사용하는 CPU의 수입니다.
)

## 이 코드는 KLUE NLI 데이터셋을 가져와서 학습용 데이터셋으로 만들고, 이를 기반으로 학습시 사용할 데이터 로더를 준비하는 과정


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
## 검증(validation) 또는 테스트용 데이터셋 및 데이터 로더를 준비하는 과정을 설명

val_dataset = ClassificationDataset(
    # 검증 또는 테스트용 데이터셋을 만듭니다.
    args=args,
    # 설정 값을 담고 있는 객체입니다.
    corpus=corpus,
    # KLUE NLI 데이터셋 객체입니다.
    tokenizer=tokenizer,
    # BERT 토크나이저입니다
    mode="test",
    # 이 데이터셋이 테스트용임을 나타냅니다.
)
val_dataloader = DataLoader(
    # 테스트용 데이터 로더를 설정합니다.
    val_dataset,
    # 테스트용 데이터셋입니다.
    batch_size=args.batch_size,
    # 한 번에 처리할 데이터의 양입니다.
    sampler=SequentialSampler(val_dataset),
    # 데이터를 순차적으로 선택하는 방식입니다. 테스트 또는 검증 과정에서는 보통 순차적으로 데이터를 처리합니다.
    collate_fn=nlpbook.data_collator,
    # 데이터를 한 번에 모아주는 함수입니다.
    drop_last=False,
    # 마지막에 남은 데이터도 사용하도록 설정합니다.
    num_workers=args.cpu_workers,
    # 데이터를 로드할 때 사용하는 CPU의 수입니다.
)

## 이 코드는 KLUE NLI 데이터셋을 기반으로 검증 또는 테스트용 데이터셋을 만들고, 그 데이터셋을 사용하여 테스트 시 사용할 데이터 로더를 준비



INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. se

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 쌍 분류를 수행할 모델을 초기화합니다.

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
# transformers 라이브러리에서 BERT 모델의 설정(BertConfig)과 시퀀스 분류 작업을 위한 BERT 모델(BertForSequenceClassification)을 가져옵니다.

pretrained_model_config = BertConfig.from_pretrained(
# BertConfig의 from_pretrained 메서드를 사용해 사전 학습된 모델의 설정을 가져옵니다.

    args.pretrained_model_name,
    # 사용할 사전 학습된 모델의 이름입니다. 여기서는 앞서 설정한 "beomi/kcbert-base"가 사용됩니다.

    num_labels=corpus.num_labels,
    # 분류 작업에서 예측할 레이블의 수를 설정합니다. 이는 corpus.num_labels를 통해 KLUE NLI 데이터셋에서 사용되는 레이블의 수를 가져와서 설정합니다.
)

##  pretrained_model_config에는 "beomi/kcbert-base" 모델의 기본 설정에, KLUE NLI 데이터셋의 레이블 수에 해당하는 num_labels 설정이 추가된 형태로 저장됩니다. 이 설정은 다음 단계에서 BERT 분류 모델을 생성할 때 사용됩니다.

In [ ]:
## 이 코드는 transformers 라이브러리의 BertForSequenceClassification을 사용하여 시퀀스 분류를 위한 BERT 모델을 초기화합니다.

model = BertForSequenceClassification.from_pretrained(
        # BertForSequenceClassification의 from_pretrained 메서드를 사용하여 사전 학습된 모델을 기반으로 시퀀스 분류 모델을 초기화합니다.
        args.pretrained_model_name,
        # 사용할 사전 학습된 모델의 이름입니다. 여기서는 앞서 설정한 "beomi/kcbert-base"를 사용합니다.
        config=pretrained_model_config,
        # 모델 설정입니다. 이 설정에는 앞서 정의한 BERT 모델의 기본 설정과 KLUE NLI 데이터셋의 레이블 수가 포함되어 있습니다.
)

## 결과적으로, model 변수에는 "beomi/kcbert-base"를 기반으로 한 시퀀스 분류용 BERT 모델이 초기화됩니다. 이 모델은 다음 단계에서 학습 및 평가에 사용됩니다.

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 학습 준비
Task와 Trainer를 준비합니다.

In [ ]:
## 이 코드는 ratsnlp 라이브러리를 사용하여 분류 작업을 위한 작업 객체를 초기화합니다.

from ratsnlp.nlpbook.classification import ClassificationTask
# 모듈 임포트: ratsnlp 라이브러리에서 분류 작업을 위한 ClassificationTask 클래스를 가져옵니다.

task = ClassificationTask(model, args)
# 분류 작업 객체 초기화: ClassificationTask 클래스의 생성자를 호출하여 분류 작업 객체를 초기화합니다.
# 주요 파라미터: 1. model, 앞서 초기화한 시퀀스 분류용 BERT 모델입니다. 2. args: 설정값을 담고 있는 객체입니다. 이 객체는 학습, 평가, 모델 저장 경로 등의 다양한 설정을 포함하고 있습니다.


## 결과적으로 'task' 변수에는 분류 작업을 위한 객체가 저장됨.

In [ ]:
## 이 코드는 ratsnlp 라이브러리의 nlpbook 모듈을 사용하여 트레이너 객체를 가져옵니다.
##  trainer 객체는 학습과 평가를 관리하고 실행하는 역할을 합니다. 다시 말해, 모델을 학습시키거나 검증 데이터를 사용하여 모델을 평가하는 데 필요한 여러 작업을 트레이너 객체가 수행합니다.

trainer = nlpbook.get_trainer(args)
# 트레이너 객체 초기화 실행
# nlpbook 모듈의 get_trainer 함수를 사용하여 트레이너 객체를 가져옵니다.
# args: 설정값을 담고 있는 객체입니다. 이 객체는 학습, 평가, 모델 저장 경로 등의 다양한 설정을 포함하고 있습니다.

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-paircls`)에 저장됩니다.

In [ ]:
## 이 코드는 앞서 초기화한 'trainer'객체를 사용하여 분류 모델의 학습 및 검증을 수행하는 역할을 함.
## 학습(fit) 메서드는 다음과 같은 작업을 수행합니다.
## 학습 데이터를 사용하여 모델을 학습시킵니다. 여기서는 'train_dataloader'에서 제공하는 데이터를 사용합니다.
## 학습 중간마다 'val_dataloader'에서 제공하는 검증 데이터를 사용하여 모델의 성능을 검증합니다.
## 모델의 성능이 개선되면 해당 모델을 저장하거나 필요한 경우 추가 작업을 수행합니다.
## 결과적으로, 이 'fit'메서드는 설정된 에폭 수 만큼 모델의 학습과 검증을 반복하여 모델의 성능을 최적화합니다.


trainer.fit(
    # 'fit' 메서드는 모델의 학습 및 검증을 수행하는 메서드
    task,
    # task: 분류 작업 객체입니다. 이 객체에는 모델과 학습, 평가를 위한 다양한 설정이 포함되어 있습니다.
    train_dataloaders=train_dataloader,
    # 학습 데이터를 제공하는 데이터 로더입니다. 이 데이터 로더는 학습 데이터를 모델에 공급하는 역할을 합니다.
    val_dataloaders=val_dataloader,
    # 검증 데이터를 제공하는 데이터 로더입니다. 학습 중간마다 모델의 성능을 검증하기 위해 사용됩니다.
)



/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-paircls exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
-----------------------------------------------------

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]